# 12-4 Multimodal Transformers
Transformersは、当初系列的な言語情報を処理するためのRNNの代替として発展してきましたが、深層学習のおおよそほとんどすべての分野に広がるにいたりました。CNNのような等価性と局所性を強く仮定するモデルとは対照的に、Transformersは入力に対し極めて少ない想定を行うため汎用モデルであることが示されています。この汎用性の高さのために、Transformersは文章、画像、動画、点群、音声データを含む様々な領域でState-of-the-artの性能を達成し、識別的、生成的応用のために用いられています。Transformer層の核を成す設計は歴史的にも目的的にも比較的一貫しており、それゆえ自然言語処理以外の分野におけるTransformersの使用を可能にしたカギとなった革新性は大部分が入力と出力の表現そして符号化にあると言える。
一つの設計を用いる一つの大きな利点は、多種多様なデータを処理することができるというものであり複数の計算を比較的一貫して行うことができるということにある。この文脈で、「複数」とは出力あるいは入力の形式として2つ以上のデータを組み合わせる、という意味で用いている。例えば、文字入力から画像を生成したいと思う時や、カメラ、レーダー、マイクなどの情報を組み合わせてロボットを操作したいと思うときなどが該当する。特筆すべき点は、入力をトークン化し出力を復号化する際、Transformerを用いることは効果的であることが多いという点だ。

## 12-4-1 Vison Transformers
Transformersの応用はコンピュータービジョンの分野で大きな成功を収めており、いくつものタスクでState-of-the-artの性能を達成している。識別的タスクにおいて最も一般的になされる選択は一般的なTransformerの符号化であり、画像の分野においてこれはVisionTransformer,ViTとして知られている。Transformerを用いる時、我々は入力画像をトークンの形に変換する方法を決めねばならないが、最も簡便な方法はそれぞれのピクセルをトークンとして用い線形に展開するということである。しかし、一般的なTransformerの埋め込みに必要なメモリは入力トークンの数に比例して二次関数的に増大していくため、一般に実現不可能とされる。代わりに、トークン化に最も一般的に用いられる手法は同じサイズのパッチに分割するというものである。画像の高さ、幅、チャンネル数がH,W,Cで与えられ、画像が$\boldsymbol{x}\in \mathbb{R}^{H \times W \times C}$で与えられるとする。（チャンネル数Ｃとは、R,G,Bの色とされる）それぞれの画像が重なることなく$P \times P$のパッチに分割される（$P=16$が最も一般的に用いられる。）そしてこれらが平らに並べられ一次元のベクトルとなり、$\boldsymbol{x}_p \in \mathbb{R}^{N\times(P^2C)},N=HW/P^2$となるような表現が、一枚の画像に対して得られる。

## 12-4-3 Audio data
Foundations of deep learning の12.4.3 Audio dataの解説を行います。音とは、空気の圧力変動として信号波形の形で保存されます。信号波形を直接的にDLモデルの入力として用いることは可能ではありますが、実用上は前処理としてMel spectrogramに通す、ということが一般的に用いられます。
transformerは、特に音声分類のために用いられることが多いです。モデルの性能を評価するために、Audioset Datasetというデータセットがよく用いられます。これは、各音声波形に対し「車」「動物」「笑い声」といったラベルが割り振られており、何らかの音声波形を通すことによってそのクラスを予測するといったものになります。Transformer以前はmel spectrogramを画像として扱い、CNNの入力として利用するというアプローチがとられていました。Transformerの利点の一つとして長い時間での依存関係・相関をとらえることができるという特徴があり、これが音声データを扱う上で非常に重要になってきます。
ここでは、mel spectrogramはまず画像として扱われ、トークンという一つの単位として分類されます。ViT(Vision Transformers)と同様の仕組みで画像をパッチに分割し、それらを一列に並べます。この場合、256の長さに並び替えられます。位置情報がこれらトークンに追加され、TransformerのEncoder層に通されます。最後のtransformer層からの出力トークンは、線形層によって複号され、ソフトマックス活性化関数を通すことによって出力が得られます。これらすべてのモデルは、クロスエントロピー誤差関数を用いることによってEnd-to-Endに訓練を行うことができます。
以下は、これらの性能比較を行うためにCNNとTransformerモデルを実装したものです。注意として、これらの実験は再現不可能である可能性があります。ViTに関しては、1080Tiに1日程度計算させる（epoch=200）ことによって良いモデルを作成することができるという計算効率の良さは証明済ですが、本稿で紹介されているASTモデルに関しては超高性能なコンピュータを使用している可能性があり、再現の目途が立っていません。
しかし、紹介されている論文(Gong,Chung,and Glass,2021)を見る限りアーキテクチャはViTに酷似しており、データの取り扱いも類似のものが見て取れましたのでViTでも同様の研究は再現可能だと考え実行を行っております。この訓練フェーズはAlvisという北欧でのリモートコンピューティングサービスを使用して、V100に計算を実行させています。